<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Getting started tutorial

In [1]:
from pyspark.sql import SparkSession

def calculate_red_violations(data_source, output_uri):
    """
    Processes sample food establishment inspection data and queries the data to find the top 10 establishments
    with the most Red violations from 2006 to 2020.

    :param data_source: The URI of your food establishment data CSV, such as 's3://DOC-EXAMPLE-BUCKET/food-establishment-data.csv'.
    :param output_uri: The URI where output is written, such as 's3://DOC-EXAMPLE-BUCKET/restaurant_violation_results'.
    """
    with SparkSession.builder.appName("Calculate Red Health Violations").getOrCreate() as spark:
        # Load the restaurant violation CSV data
        if data_source is not None:
            restaurants_df = spark.read.option("header", "true").csv(data_source)

        # Create an in-memory DataFrame to query
        restaurants_df.createOrReplaceTempView("restaurant_violations")

        # Create a DataFrame of the top 10 restaurants with the most Red violations
        top_red_violation_restaurants = spark.sql("""SELECT name, count(*) AS total_red_violations 
          FROM restaurant_violations 
          WHERE violation_type = 'RED' 
          GROUP BY name 
          ORDER BY total_red_violations DESC LIMIT 10""")

        # Write the results to the specified output URI
        top_red_violation_restaurants.write.option("header", "true").mode("overwrite").csv(output_uri)

# Set the values for data_source and output_uri
data_source = 's3://gelcawsemr/food_establishment_data.csv'
output_uri = 's3://gelcawsemr/restaurant_violation_results'

# Call the function with the provided values
calculate_red_violations(data_source, output_uri)


VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,application_1701743246151_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Programme to uncompress the archives

In [ ]:
import boto3
import tarfile
import bz2
import os

# Set up AWS credentials
aws_access_key_id = 'YOUR_ACCESS_KEY_ID'
aws_secret_access_key = 'YOUR_SECRET_ACCESS_KEY'
region_name = 'YOUR_REGION_NAME'

# Set up S3 client
s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

# Set up EMR client
emr = boto3.client('emr', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

# Set up the job flow
job_flow_id = 'YOUR_JOB_FLOW_ID'

# Set up the source and destination S3 bucket names
source_bucket_name = 'YOUR_SOURCE_BUCKET_NAME'
destination_bucket_name = 'YOUR_DESTINATION_BUCKET_NAME'

# Set up the S3 bucket and file paths
tar_file_key = 'path/to/your/tar/file.tar'

# Fetch the .tar file from the source S3 bucket
s3.download_file(source_bucket_name, tar_file_key, 'file.tar')

# Uncompress the .tar file
with tarfile.open('file.tar', 'r') as tar:
    tar.extractall()

# Iterate over the extracted files
for root, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.bz2'):
            # Uncompress each .bz2 file
            with bz2.open(os.path.join(root, file), 'rb') as bz_file:
                uncompressed_data = bz_file.read()
                # Process the uncompressed data (e.g., write to a new file, parse JSON, etc.)
                # Your processing logic goes here
                
                # Get the relative path of the file within the directory tree
                relative_path = os.path.relpath(os.path.join(root, file), '.')

                # Upload the processed file to the destination S3 bucket with the same directory tree structure
                destination_key = os.path.join(relative_path, file)
                s3.put_object(Body=uncompressed_data, Bucket=destination_bucket_name, Key=destination_key)

# Terminate the EMR cluster
emr.terminate_job_flows(JobFlowIds=[job_flow_id])

## Sample code to parse the tweets files to a dataframe over Amazon EMR

In [ ]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("Tweet Analysis") \
    .getOrCreate()

# Read the tweets file from S3 and create a dataframe
tweets_df = spark.read.json("s3://your-bucket/tweets.json")

# Filter the dataframe to extract the desired data
filtered_df = tweets_df.filter(tweets_df['text'].contains("your-filter-keyword"))

# Show the filtered dataframe
filtered_df.show()

# snscrape format

In [49]:
import pandas as pd

# Read the JSON file into a DataFrame
df1 = pd.read_json('snscrape.json', orient='records', lines=True)

In [51]:
df1

,_type,url,date,rawContent,renderedContent,id,user,replyCount,retweetCount,likeCount,...,retweetedTweet,quotedTweet,inReplyToTweetId,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,card
0,snscrape.modules.twitter.Tweet,https://twitter.com/AmarilsaPi/status/10907603...,2019-01-30 23:55:17+00:00,@estudioi Não estou preocupada com o custo fin...,@estudioi Não estou preocupada com o custo fin...,1090760391850934281,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,1.090622e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,NaN,None
1,snscrape.modules.twitter.Tweet,https://twitter.com/FranciscoSenaDF/status/109...,2019-01-30 23:54:12+00:00,@Antonbaptista @folha Venezuela é ditadura gen...,@Antonbaptista @folha Venezuela é ditadura gen...,1090760120911491073,"{'_type': 'snscrape.modules.twitter.User', 'us...",3,0,1,...,NaN,None,1.090757e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,NaN,None
2,snscrape.modules.twitter.Tweet,https://twitter.com/eduardolm171/status/109075...,2019-01-30 23:51:42+00:00,Bolsonaro se encontrou com presidente genocida...,Bolsonaro se encontrou com presidente genocida...,1090759491627503616,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,NaN,None,"[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,NaN,None
3,snscrape.modules.twitter.Tweet,https://twitter.com/VPFac/status/1090756459099...,2019-01-30 23:39:39+00:00,"@PolitzOficial È uma questão humanitária, sim,...","@PolitzOficial È uma questão humanitária, sim,...",1090756459099537408,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,1.090375e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,[chorapetralha],NaN,None
4,snscrape.modules.twitter.Tweet,https://twitter.com/DrReynaldo/status/10907552...,2019-01-30 23:34:44+00:00,@monicabergamo A Sra pensa q é correto esse ge...,@monicabergamo A Sra pensa q é correto esse ge...,1090755218566975488,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,2,...,NaN,None,1.090623e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,snscrape.modules.twitter.Tweet,https://twitter.com/teodeiocomamor/status/1090...,2019-01-29 21:59:00+00:00,"@gleisi Porque é de um corrupto, assassino, g...","@gleisi Porque é de um corrupto, assassino, g...",1090368739349094405,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,...,NaN,None,1.090367e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,NaN,None
196,snscrape.modules.twitter.Tweet,https://twitter.com/Melo_Guilherme/status/1090...,2019-01-29 21:58:48+00:00,"@MBLivre Me lembra um genocida, chamado Mandel...","@MBLivre Me lembra um genocida, chamado Mandel...",1090368689793351681,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,NaN,None,1.090262e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,NaN,None
197,snscrape.modules.twitter.Tweet,https://twitter.com/andersonino/status/1090364...,2019-01-29 21:42:40+00:00,"@gwannael @folha O que é o que é, tem cara de ...","@gwannael @folha O que é o que é, tem cara de ...",1090364630176878593,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,0,...,NaN,None,1.090351e+18,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",None,None,None,NaN,None
198,snscrape.modules.twitter.Tweet,https://twitter.com/renata_passari/status/1090...,2019-01-29 21:08:40+00:00,@GloboNews @MarceloLins68 Vão dizer que foi go...,@GloboNews @MarceloLins68 Vão dizer que foi g

# Internet Archive format

In [53]:
import pandas as pd

# Read the JSON file into a DataFrame
df2 = pd.read_json('intarch.json', orient='records', lines=True)

/tmp/ipykernel_754/2891185936.py:4: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df2 = pd.read_json('intarch.json', orient='records', lines=True)
/tmp/ipykernel_754/2891185936.py:4: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df2 = pd.read_json('intarch.json', orient='records', lines=True)


In [55]:
df2

,created_at,id,id_str,text,display_text_range,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,timestamp_ms,retweeted_status,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_tweet,delete
0,2019-01-01 07:29:00+00:00,1.080003e+18,1.080003e+18,@supr_dorapo 最新のゼルダやってないけどゼルダも好きですﾋﾟｭｰﾋﾟｭｰﾋﾟｭﾋ...,"[13, 58]","<a href=""http://twitter.com/download/android"" ...",0.0,1.080003e+18,1.080003e+18,2.787406e+09,...,2019-01-01 07:28:07.168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01 07:29:00+00:00,1.080003e+18,1.080003e+18,そうだよ、僕がピーマンだよ,NaN,"<a href=""http://twittbot.net/"" rel=""nofollow"">...",0.0,NaN,NaN,NaN,...,2019-01-01 07:28:07.168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01 07:29:00+00:00,1.080003e+18,1.080003e+18,@subaru_yamamoto 勝ち退くが利って、それ得意なひとTwitterにたくさんい...,"[17, 66]","<a href=""http://twitter.com/download/iphone"" r...",0.0,1.080002e+18,1.080002e+18,3.955496e+09,...,2019-01-01 07:28:07.168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01 07:29:00+00:00,1.080003e+18,1.080003e+18,RT @PPGcnjp: 初もうで😊 https://t.co/NOfd4q1ro5,NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,NaN,...,2019-01-01 07:28:07.168,{'created_at': 'Tue Jan 01 07:24:00 +0000 2019...,"{'media': [{'id': 1076011587624873984, 'id_str...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01 07:29:00+00:00,1.080003e+18,1.080003e+18,RT @DJ_GINTA_NEW: 【お正月の歌作ったよん🎍】 https://t.co/e...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,NaN,...,2019-01-01 07:28:07.168,{'created_at': 'Tue Jan 01 07:22:06 +0000 2019...,"{'media': [{'id': 1080001090727247872, 'id_str...",0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2019-01-01 07:29:04+00:00,1.080003e+18,1.080003e+18,@Fams000 @Rouguii77 +mention inch'allah bonne ...,"[20, 59]","<a href=""http://twitter.com/download/android"" ...",0.0,1.079895e+18,1.079895e+18,2.878622e+09,...,2019-01-01 07:28:07.168,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,2019-01-01 07:29:04+00:00,1.080003e+18,1.080003e+18,RT @Sphenodontiaart: hope to god they kiss for...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,NaN,...,2019-01-01 07:28:07.168,{'created_at': 'Mon Dec 31 06:09:07 +0000 2018...,"{'media': [{'id': 1079620371505065984, 'id_str...",0.0,NaN,NaN,NaN,NaN,NaN,NaN
197,2019-01-01 07:29:04+00:00,1.080003e+18,1.080003e+18,RT @fuzakefactory: 明けましておめでとうございます🐗\n新年お年玉企画🎉\...,NaN,"<a href=""http://twitter.com/download/iphone"" r...",0.0,NaN,NaN,NaN,...,2019-01-01 07:28:07.168,{'created_at': 'Tue Jan 01 06:45:32 +0000 2019...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,2019-01-01 07:29:04+00:00,1.080003e+18,1.080003e+18,Feliz año nuevo! Happy new year! #2019 @ Nuevo...,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",0.0,NaN,NaN,NaN,...,2019-01-01 07:28:07.168,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
